In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy.typing as npt
import matplotlib.pyplot as plt

#TODO
- fazer plot da legenda ficar sempre no canto superior esquerdo
- por titulo na legenda dizendo qual variṕavel esta avriando
- pegar chave e valor de valores fixos de forma mais elegante

In [2]:
def plot_empirical_error_convergence(loss: npt.NDArray[np.float_], nn_config: dict, not_fixed_param: str, annotate: bool = False) -> None:
    sns.set(style="darkgrid", color_codes=True, rc={"figure.figsize": (8, 5)})
    ax = sns.lineplot(data=loss, markers=True, marker="o", label=f"{nn_config[not_fixed_param]}")

    del nn_config[not_fixed_param]
    keys = list(nn_config.keys())
    vls = list(nn_config.values())

    plt.xticks(np.arange(len(loss)))
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(title=not_fixed_param, fontsize=10)
    plt.title(f"Convergence of empirical error\n{keys[0]} = {vls[0]} , {keys[1]} = {vls[1]}")
    if annotate:
        for i, j in enumerate(loss):
            ax.annotate(str(np.round(j, 2)), xy=(i, j), xytext=(i, j + 0.02))

In [3]:
def plot_fixed(results: pd.DataFrame, hidden_layer_size: np.int_ = None, batch_size: np.int_ = None, learning_rate: np.float_ = None, annotate: bool = False) -> None:
    if hidden_layer_size is None and batch_size is None and learning_rate is None:
        raise "Especifique quais métricas devem ser fixadas"
    not_fixed: str = ""
    
    if hidden_layer_size and batch_size:
        aux: pd.DataFrame = results.query(f"hidden_layer_size == {hidden_layer_size} and batch_size == {batch_size}")[["learning_rate", "history"]]
        not_fixed = "learning_rate"
        for i in range(aux.shape[0]):
            e = aux.iloc[i]
            plot_empirical_error_convergence(loss=e.history["loss"], nn_config={"hidden_layer_size": hidden_layer_size, "batch_size": batch_size, "learning_rate": e.learning_rate}, not_fixed_param=not_fixed, annotate=annotate)
    
    if hidden_layer_size and learning_rate:
        aux: pd.DataFrame = results.query(f"hidden_layer_size == {hidden_layer_size} and learning_rate == {learning_rate}")[["batch_size", "history"]]
        not_fixed = "batch_size"
        for i in range(aux.shape[0]):
            e = aux.iloc[i]
            plot_empirical_error_convergence(loss=e.history["loss"], nn_config={"hidden_layer_size": hidden_layer_size, "batch_size": e.batch_size, "learning_rate": learning_rate}, not_fixed_param=not_fixed, annotate=annotate)
    
    if learning_rate and batch_size:
        aux: pd.DataFrame = results.query(f"batch_size == {batch_size} and learning_rate == {learning_rate}")[["hidden_layer_size", "history"]]
        not_fixed = "hidden_layer_size"
        for i in range(aux.shape[0]):
            e = aux.iloc[i]
            plot_empirical_error_convergence(loss=e.history["loss"], nn_config={"hidden_layer_size": e.hidden_layer_size, "batch_size": batch_size, "learning_rate": learning_rate}, not_fixed_param=not_fixed, annotate=annotate)

In [4]:
results: pd.DataFrame = pd.read_json("data/results.json")
results.nlargest(columns=["accuracy_score"], n=10)

,accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,random_state_seed,test_size,hidden_layer_size,batch_size,learning_rate,history
27,0.931333,"[0.9307692307692301, 0.9696969696969691, 0.933...","[0.968, 0.9815950920245391, 0.9130434782608691...","[0.9490196078431371, 0.9756097560975611, 0.923...","[[121, 0, 0, 0, 1, 2, 0, 0, 1, 0], [0, 160, 0,...",42,0.3,100,20,1.5,"{'loss': [0.836005508899688, 0.331558793783187..."
28,0.930667,"[0.9179104477611941, 0.9523809523809521, 0.919...","[0.984, 0.9815950920245391, 0.9130434782608691...","[0.949806949806949, 0.9667673716012081, 0.9163...","[[123, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 160, 0,...",42,0.3,100,20,1.0,"{'loss': [0.8691549301147461, 0.36839935183525..."
15,0.930000,"[0.9179104477611941, 0.9578313253012041, 0.940...","[0.984, 0.9754601226993861, 0.9130434782608691...","[0.949806949806949, 0.966565349544073, 0.92647...","[[123, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 159, 0,...",42,0.3,50,20,1.5,"{'loss': [0.7906588912010191, 0.33696186542510..."
16,0.928667,"[0.9179104477611941, 0.9575757575757571, 0.927...","[0.984, 0.9693251533742331, 0.9202898550724631...","[0.949806949806949, 0.9634146341463411, 0.9236...","[[123, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 158, 0,...",42,0.3,50,20,1.0,"{'loss': [0.846143841743469, 0.372355490922927..."
30,0.925333,"[0.9307692307692301, 0.952095808383233, 0.8951...","[0.968, 0.9754601226993861, 0.9275362318840581...","[0.9490196078431371, 0.9636363636363631, 0.911...","[[121, 0, 1, 0, 0, 1, 1, 0, 1, 0], [0, 159, 0,...",42,0.3,100,50,1.5,"{'loss': [1.184221029281616, 0.438366383314132..."
31,0.925333,"[0.9242424242424241, 0.952095808383233, 0.9130...","[0.976, 0.9754601226993861, 0.9130434782608691...","[0.9494163424124511, 0.9636363636363631, 0.913...","[[122, 0, 1, 0, 0, 0, 0, 0, 2, 0], [0, 159, 0,...",42,0.3,100,50,1.0,"{'loss': [1.178818583488464, 0.471435487270355..."
4,0.924667,"[0.9453125, 0.9695121951219511, 0.917293233082...","[0.968, 0.9754601226993861, 0.884057971014492,...","[0.956521739130434, 0.9724770642201831, 0.9003...","[[121, 0, 0, 0, 0, 2, 1, 0, 1, 0], [0, 159, 0,...",42,0.3,26,20,1.0,"{'loss': [0.8392609953880311, 0.37692862749099..."
18,0.922667,"[0.9318181818181811, 0.9461077844311371, 0.920...","[0.984, 0.9693251533742331, 0.9202898550724631...","[0.957198443579766, 0.9575757575757571, 0.9202...","[[123, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 158, 0,...",42,0.3,50,50,1.5,"{'loss': [1.058859825134277, 0.429420292377471..."
19,0.918000,"[0.9384615384615381, 0.951807228915662, 0.9191...","[0.976, 0.9693251533742331, 0.905797101449275,...","[0.956862745098039, 0.9604863221884491, 0.9124...","[[122, 0, 1, 0, 0, 0, 0, 0, 2, 0], [0, 158, 0,...",42,0.3,50,50,1.0,"{'loss': [1.085147976875305, 0.488063365221023..."
3,0.915333,"[0.9248120300751881, 0.951807228915662, 0.9253...","[0.984, 0.9693251533742331, 0.898550724637681,...","[0.9534883720930231, 0.9604863221884491, 0.911...","[[123, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 158, 0,...",42,0.3,26,20,1.5,"{'loss': [0.8034397959709161, 0.37639909982681..."


In [5]:
results.nlargest(columns=["accuracy_score"], n=10)

,accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,random_state_seed,test_size,hidden_layer_size,batch_size,learning_rate,history
27,0.931333,"[0.9307692307692301, 0.9696969696969691, 0.933...","[0.968, 0.9815950920245391, 0.9130434782608691...","[0.9490196078431371, 0.9756097560975611, 0.923...","[[121, 0, 0, 0, 1, 2, 0, 0, 1, 0], [0, 160, 0,...",42,0.3,100,20,1.5,"{'loss': [0.836005508899688, 0.331558793783187..."
28,0.930667,"[0.9179104477611941, 0.9523809523809521, 0.919...","[0.984, 0.9815950920245391, 0.9130434782608691...","[0.949806949806949, 0.9667673716012081, 0.9163...","[[123, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 160, 0,...",42,0.3,100,20,1.0,"{'loss': [0.8691549301147461, 0.36839935183525..."
15,0.930000,"[0.9179104477611941, 0.9578313253012041, 0.940...","[0.984, 0.9754601226993861, 0.9130434782608691...","[0.949806949806949, 0.966565349544073, 0.92647...","[[123, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 159, 0,...",42,0.3,50,20,1.5,"{'loss': [0.7906588912010191, 0.33696186542510..."
16,0.928667,"[0.9179104477611941, 0.9575757575757571, 0.927...","[0.984, 0.9693251533742331, 0.9202898550724631...","[0.949806949806949, 0.9634146341463411, 0.9236...","[[123, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 158, 0,...",42,0.3,50,20,1.0,"{'loss': [0.846143841743469, 0.372355490922927..."
30,0.925333,"[0.9307692307692301, 0.952095808383233, 0.8951...","[0.968, 0.9754601226993861, 0.9275362318840581...","[0.9490196078431371, 0.9636363636363631, 0.911...","[[121, 0, 1, 0, 0, 1, 1, 0, 1, 0], [0, 159, 0,...",42,0.3,100,50,1.5,"{'loss': [1.184221029281616, 0.438366383314132..."
31,0.925333,"[0.9242424242424241, 0.952095808383233, 0.9130...","[0.976, 0.9754601226993861, 0.9130434782608691...","[0.9494163424124511, 0.9636363636363631, 0.913...","[[122, 0, 1, 0, 0, 0, 0, 0, 2, 0], [0, 159, 0,...",42,0.3,100,50,1.0,"{'loss': [1.178818583488464, 0.471435487270355..."
4,0.924667,"[0.9453125, 0.9695121951219511, 0.917293233082...","[0.968, 0.9754601226993861, 0.884057971014492,...","[0.956521739130434, 0.9724770642201831, 0.9003...","[[121, 0, 0, 0, 0, 2, 1, 0, 1, 0], [0, 159, 0,...",42,0.3,26,20,1.0,"{'loss': [0.8392609953880311, 0.37692862749099..."
18,0.922667,"[0.9318181818181811, 0.9461077844311371, 0.920...","[0.984, 0.9693251533742331, 0.9202898550724631...","[0.957198443579766, 0.9575757575757571, 0.9202...","[[123, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 158, 0,...",42,0.3,50,50,1.5,"{'loss': [1.058859825134277, 0.429420292377471..."
19,0.918000,"[0.9384615384615381, 0.951807228915662, 0.9191...","[0.976, 0.9693251533742331, 0.905797101449275,...","[0.956862745098039, 0.9604863221884491, 0.9124...","[[122, 0, 1, 0, 0, 0, 0, 0, 2, 0], [0, 158, 0,...",42,0.3,50,50,1.0,"{'loss': [1.085147976875305, 0.488063365221023..."
3,0.915333,"[0.9248120300751881, 0.951807228915662, 0.9253...","[0.984, 0.9693251533742331, 0.898550724637681,...","[0.9534883720930231, 0.9604863221884491, 0.911...","[[123, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 158, 0,...",42,0.3,26,20,1.5,"{'loss': [0.8034397959709161, 0.37639909982681..."
